In [1]:
import pandas as pd
from time import sleep
import os

import xlwings as xw

# from datetime import datetime

# import calendar
# import locale

In [2]:
exportar = True

abrir_pdfs = True

In [3]:
trimestres = {
    1: ['janeiro', 'fevereiro', 'março'],
    2: ['abril', 'maio', 'junho'],
    3: ['julho', 'agosto', 'setembro'],
    4: ['outubro', 'novembro', 'dezembro']
}

nome_campanha = 'NATAL PREMIADO'


trimestre_atual = pd.Timestamp.now().quarter
meses_do_trimestre_atual = list(map(lambda x: x.upper(), trimestres[trimestre_atual]))
meses_do_trimestre_atual

['OUTUBRO', 'NOVEMBRO', 'DEZEMBRO']

#### <span style="color: red;">**ARQUIVOS AUXILIARES**</span>

In [4]:
caminho = r'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\ARQUIVOS AUXILIARES\REGIOES.xlsx'
df_info_regioes = pd.read_excel(caminho)

df_info_regioes.dropna(inplace=True)

areas = df_info_regioes['AREA'].unique()

print(areas)
display(df_info_regioes)

['AREA MG' 'AREA SP' 'AREA CENTRO OESTE']


,REGIAO,AREA,SUPERVISOR
0,RG UBERLANDIA,AREA MG,RODRIGO VIEIRA
1,RG PATOS,AREA MG,IRIS ROBERT
2,RG UBERABA,AREA MG,PAULO MIRANDA
3,RG BH,AREA MG,HELIO CAMPOS
4,RG MONTES CLAROS,AREA MG,RONEY AGUIAR
5,RG GOV VALADARES,AREA MG,RANGEL OLIVEIRA
6,RG SUL DE MINAS,AREA MG,EDER GODOI
7,RG JUIZ DE FORA,AREA MG,RODRIGO MACHADO
8,RG DIVINOPOLIS,AREA MG,MAURO LOBATO
9,RG RIBEIRAO,AREA SP,THIAGO REIS


##### **METAS**

In [5]:
df_metas_representates = pd.read_excel(rf'arq_excel\METAS - CAMPANHA {nome_campanha}.xlsx', sheet_name='METAS-RP', dtype={'COD': str})
display(df_metas_representates.head(3))

supervisores = df_metas_representates['SUPERVISOR'].unique()
print(supervisores)

vendedores_com_metas = df_metas_representates['COD'].unique()
print(len(vendedores_com_metas))

,REGIAO,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META TRI
0,RG BH,HELIO CAMPOS,377,EDUARDO BARBOSA,BARBACENA,3300
1,RG BH,HELIO CAMPOS,402,EULER ROMUALDO,OURO PRETO,3500
2,RG BH,HELIO CAMPOS,567,ROSILENE MARIA,CURVELO,700


['HELIO CAMPOS' 'AMANCIO JUNIO' 'CARLOS FERNANDES' 'MAURO LOBATO'
 'MARCELO TOMAZELI' 'LEANDRO QUEIROZ' 'RANGEL OLIVEIRA' 'RODRIGO MACHADO'
 'RONEY AGUIAR' 'IRIS ROBERT' 'THIAGO REIS' 'DOUGLAS CORSINO'
 'LEANDRO JOSE' 'EDER GODOI' 'PAULO MIRANDA' 'RODRIGO VIEIRA']
113


In [6]:
df_metas_regioes = pd.read_excel(rf'arq_excel\METAS - CAMPANHA {nome_campanha}.xlsx', sheet_name='METAS-RG')
df_metas_regioes = df_metas_regioes.query('REGIAO != "RG BAURU"')
display(df_metas_regioes.head(3))


regioes = df_metas_regioes['REGIAO'].unique()
print(regioes)

,REGIAO,META TRI
0,RG UBERLANDIA,102000
1,RG PATOS,60000
2,RG UBERABA,57000


['RG UBERLANDIA' 'RG PATOS' 'RG UBERABA' 'RG BH' 'RG MONTES CLAROS'
 'RG GOV VALADARES' 'RG SUL DE MINAS' 'RG JUIZ DE FORA' 'RG DIVINOPOLIS'
 'RG RIBEIRAO' 'RG RIO PRETO' 'RG FRANCA' 'RG CAMPINAS' 'RG BRASILIA'
 'RG SUL DE GOIAS' 'RG GOIANIA']


##### **VENDAS**

In [7]:
df_vendas = pd.read_csv(r'arq_csv\Parcial Vendas.csv', decimal=',')

df_vendas['CodVendedor'] = df_vendas['CodVendedor'].astype(str)
df_vendas['Mês'] = df_vendas['Mês'].apply(lambda x: x.upper())
df_vendas['Região'] = df_vendas['Região'].apply(lambda x: x.upper())
# df_vendas['QtdVendida'] = df_vendas['QtdVendida'].apply(lambda x: round(x))


display(df_vendas.head(3))
display(df_vendas['QtdVendida'].sum())

meses_com_vendas = df_vendas['Mês'].unique()
# print(meses_com_vendas)


vendedores_com_resultado = df_vendas['CodVendedor'].unique()
# print(vendedores_com_resultado)

regioes_com_vendas = list(df_vendas['Região'].unique())
# print(regioes_com_vendas)

,Região,CodVendedor,Mês,Linha,QtdVendida
0,RG BRASILIA,1,OUTUBRO,Premium,7.00
1,RG UBERLANDIA,2,OUTUBRO,Premium,710.63
2,RG UBERLANDIA,3,OUTUBRO,Premium,416.98


34386.649999999994

In [8]:
display(df_vendas[df_vendas['Região'] == 'RG BH']['QtdVendida'].sum())

4931.08

#### <span style="color: red;">**REGIÃO**</span>

##### **VENDAS POR REGIAO**

In [9]:
df_vendas_por_regiao = df_vendas.loc[:, ('Região', 'Mês', 'QtdVendida')]
df_vendas_por_regiao = df_vendas_por_regiao.groupby(['Região', 'Mês'])['QtdVendida'].sum().reset_index()
df_vendas_por_regiao = df_vendas_por_regiao.query('Região != "RG BAURU"')


df_vendas_por_regiao = df_vendas_por_regiao.pivot_table(index='Região', columns='Mês', values='QtdVendida').reset_index()
df_vendas_por_regiao.columns.name = None


columns = df_vendas_por_regiao.columns


for mes in meses_do_trimestre_atual:
    if mes not in columns:
        df_vendas_por_regiao[mes] = 0


df_vendas_por_regiao = df_vendas_por_regiao[['Região'] + meses_do_trimestre_atual]
df_vendas_por_regiao.columns = ['Região'] + list(map(lambda x: f'REAL {x}', meses_do_trimestre_atual))

display(df_vendas_por_regiao.head(3))

,Região,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO
0,RG BH,4931.08,0,0
1,RG BRASILIA,151.67,0,0
2,RG CAMPINAS,174.34,0,0


##### **CAMPANHA DE REGIAO**

In [10]:
df_vendas_por_regiao = pd.merge(df_metas_regioes, df_vendas_por_regiao, left_on='REGIAO', right_on='Região', how='left')

df_vendas_por_regiao.fillna(0, inplace=True)

df_vendas_por_regiao['TOTAL TRI'] = df_vendas_por_regiao[f'REAL {meses_do_trimestre_atual[0]}'] + df_vendas_por_regiao[f'REAL {meses_do_trimestre_atual[1]}'] + df_vendas_por_regiao[f'REAL {meses_do_trimestre_atual[2]}']

df_vendas_por_regiao[r'% REAL X META'] = round(df_vendas_por_regiao['TOTAL TRI'] / df_vendas_por_regiao['META TRI'], 2)
# df_vendas_por_regiao[r'% REAL X META'] = df_vendas_por_regiao[r'% REAL X META'].apply(lambda x: round(x))

df_vendas_por_regiao = pd.merge(df_vendas_por_regiao, df_info_regioes, on='REGIAO', how='left').drop(columns=['SUPERVISOR'])
df_vendas_por_regiao.drop('Região', axis=1, inplace=True)

display(df_vendas_por_regiao.head(3))

,REGIAO,META TRI,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO,TOTAL TRI,% REAL X META,AREA
0,RG UBERLANDIA,102000,8425.55,0,0,8425.55,0.08,AREA MG
1,RG PATOS,60000,4767.73,0,0,4767.73,0.08,AREA MG
2,RG UBERABA,57000,4220.74,0,0,4220.74,0.07,AREA MG


In [11]:
for area in areas:
    df_export = df_vendas_por_regiao[df_vendas_por_regiao['AREA'] == area].reset_index(drop=True)
    df_export.drop(columns=['AREA'], inplace=True)

    index = len(df_export)

    texto = f'TOTAL {area}'
    total_meta_tri = df_export['META TRI'].sum()
    total_real_mes1 = df_export[f'REAL {meses_do_trimestre_atual[0]}'].sum()
    total_real_mes2 = df_export[f'REAL {meses_do_trimestre_atual[1]}'].sum()
    total_real_mes3 = df_export[f'REAL {meses_do_trimestre_atual[2]}'].sum()
    total_tri = df_export['TOTAL TRI'].sum()
    # real_x_meta = df_export['% REAL X META']

    df_export.loc[index] = {
        'REGIAO': texto, 
        'META TRI': total_meta_tri, 
        f'REAL {meses_do_trimestre_atual[0]}': total_real_mes1,
        f'REAL {meses_do_trimestre_atual[1]}': total_real_mes2,
        f'REAL {meses_do_trimestre_atual[2]}': total_real_mes3,
        'TOTAL TRI': total_tri
    }

    real_x_meta = round((total_tri / total_meta_tri), 2)

    df_export.loc[index, '% REAL X META'] = real_x_meta

    # df_export['% REAL X META'] = df_export['% REAL X META'].apply(lambda x: round(x))

    display(df_export.head(2))

    print(total_meta_tri)

    if exportar:
        df_export.to_excel(fr'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\CAMPANHA - {nome_campanha}\consolidados gerados - para o envio\RESULTADO REGIOES\{area}.xlsx', index=False)

,REGIAO,META TRI,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO,TOTAL TRI,% REAL X META
0,RG UBERLANDIA,102000,8425.55,0,0,8425.55,0.08
1,RG PATOS,60000,4767.73,0,0,4767.73,0.08


332700


,REGIAO,META TRI,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO,TOTAL TRI,% REAL X META
0,RG RIBEIRAO,40000,1170.28,0,0,1170.28,0.03
1,RG RIO PRETO,26000,1482.29,0,0,1482.29,0.06


99000


,REGIAO,META TRI,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO,TOTAL TRI,% REAL X META
0,RG BRASILIA,9000,151.67,0,0,151.67,0.02
1,RG SUL DE GOIAS,39000,3087.32,0,0,3087.32,0.08


61000


#### <span style="color: red;">**REPRESENTANTES**</span>

##### **VENDAS POR REPRESENTATES**

In [12]:
df_vendas_por_representante = df_vendas.loc[:, ('CodVendedor', 'Mês', 'QtdVendida')]
df_vendas_por_representante = df_vendas_por_representante.groupby(['CodVendedor', 'Mês'])['QtdVendida'].sum().reset_index()

df_vendas_por_representante = df_vendas_por_representante.pivot_table(index='CodVendedor', columns='Mês', values='QtdVendida').reset_index()
df_vendas_por_representante.columns.name = None


columns = df_vendas_por_representante.columns


# df_vendas_por_representante.fillna(0, inplace=True)


for mes in meses_do_trimestre_atual:
    if mes not in columns:
        df_vendas_por_representante[mes] = 0

      
df_vendas_por_representante = df_vendas_por_representante[['CodVendedor'] + meses_do_trimestre_atual]
df_vendas_por_representante.columns = ['CodVendedor'] + list(map(lambda x: f'REAL {x}', meses_do_trimestre_atual))


# for real_mes in list(map(lambda x: f'REAL {x}', meses_do_trimestre_atual)):
#     df_vendas_por_representante[real_mes] = df_vendas_por_representante[real_mes].apply(lambda x: round(x))

display(df_vendas_por_representante.head(2))
# df_vendas_por_representante.info()

,CodVendedor,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO
0,1,7.00,0,0
1,101,77.33,0,0


##### **CAMPANHA DE REPRESENTANTES**

In [13]:
df_vendas_por_representante = pd.merge(df_metas_representates, df_vendas_por_representante, left_on='COD', right_on='CodVendedor', how='left').drop(columns=['CodVendedor'])

df_vendas_por_representante.fillna(0, inplace=True)

df_vendas_por_representante['TOTAL TRI'] = df_vendas_por_representante[f'REAL {meses_do_trimestre_atual[0]}'] + df_vendas_por_representante[f'REAL {meses_do_trimestre_atual[1]}'] + df_vendas_por_representante[f'REAL {meses_do_trimestre_atual[2]}']

df_vendas_por_representante[r'% REAL X META'] = round(df_vendas_por_representante['TOTAL TRI'] / df_vendas_por_representante['META TRI'], 2)
# df_vendas_por_representante[r'% REAL X META'] = df_vendas_por_representante[r'% REAL X META'].apply(lambda x: round(x))
# df_vendas_por_representante[r'TOTAL TRI'] = df_vendas_por_representante[r'TOTAL TRI'].apply(lambda x: round(x))

# for real_mes in list(map(lambda x: f'REAL {x}', meses_do_trimestre_atual)):
#     df_vendas_por_representante[real_mes] = df_vendas_por_representante[real_mes].apply(lambda x: round(x))

# df_vendas_por_representante[r'% REAL X META'] = df_vendas_por_representante[r'% REAL X META']

display(df_vendas_por_representante.head(2))

,REGIAO,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META TRI,REAL OUTUBRO,REAL NOVEMBRO,REAL DEZEMBRO,TOTAL TRI,% REAL X META
0,RG BH,HELIO CAMPOS,377,EDUARDO BARBOSA,BARBACENA,3300,165.10,0.0,0.0,165.10,0.05
1,RG BH,HELIO CAMPOS,402,EULER ROMUALDO,OURO PRETO,3500,122.32,0.0,0.0,122.32,0.03


In [14]:
for regiao in regioes:
    df_export = df_vendas_por_representante[df_vendas_por_representante['REGIAO'] == regiao].reset_index(drop=True)
    df_export.drop(columns=['REGIAO'], inplace=True)

    supervisor = df_export['SUPERVISOR'][0]
    index = len(df_export)

    texto = f'TOTAL REPRESENTANTES - SUPERVISOR {supervisor}'
    total_meta_tri = df_export['META TRI'].sum()
    total_real_mes1 = df_export[f'REAL {meses_do_trimestre_atual[0]}'].sum()
    total_real_mes2 = df_export[f'REAL {meses_do_trimestre_atual[1]}'].sum()
    total_real_mes3 = df_export[f'REAL {meses_do_trimestre_atual[2]}'].sum()
    total_tri = df_export['TOTAL TRI'].sum()

    df_export.loc[index] = {
        'SUPERVISOR': texto, 
        'META TRI': total_meta_tri, 
        f'REAL {meses_do_trimestre_atual[0]}': total_real_mes1,
        f'REAL {meses_do_trimestre_atual[1]}': total_real_mes2,
        f'REAL {meses_do_trimestre_atual[2]}': total_real_mes3,
        'TOTAL TRI': total_tri
    }

    real_x_meta = round((total_tri / total_meta_tri), 2)

    df_export.loc[index, '% REAL X META'] = real_x_meta



    ####### ULTIMA LINHA #######

    df_valores_por_regiao = df_vendas_por_regiao[df_vendas_por_regiao['REGIAO'] == regiao]

    texto = 'TOTAIS DA REGIÃO'
    total_meta_tri = df_valores_por_regiao['META TRI'].values[0]
    total_real_mes1 = df_valores_por_regiao[f'REAL {meses_do_trimestre_atual[0]}'].values[0]
    total_real_mes2 = df_valores_por_regiao[f'REAL {meses_do_trimestre_atual[1]}'].values[0]
    total_real_mes3 = df_valores_por_regiao[f'REAL {meses_do_trimestre_atual[2]}'].values[0]
    total_tri = df_valores_por_regiao['TOTAL TRI'].values[0]
    real_x_meta = df_valores_por_regiao['% REAL X META'].values[0]



    df_export.loc[index + 1] = {
        'SUPERVISOR': texto,
        'PRAÇA': regiao,
        'META TRI': total_meta_tri, 
        f'REAL {meses_do_trimestre_atual[0]}': total_real_mes1,
        f'REAL {meses_do_trimestre_atual[1]}': total_real_mes2,
        f'REAL {meses_do_trimestre_atual[2]}': total_real_mes3,
        'TOTAL TRI': total_tri,
        '% REAL X META': real_x_meta
}


    df_export = df_export.rename(columns={
        'META TRI': 'META\nTRI',
        f'REAL {meses_do_trimestre_atual[0]}': f'REAL\n{meses_do_trimestre_atual[0]}',
        f'REAL {meses_do_trimestre_atual[1]}': f'REAL\n{meses_do_trimestre_atual[1]}',
        f'REAL {meses_do_trimestre_atual[2]}': f'REAL\n{meses_do_trimestre_atual[2]}',
        'TOTAL TRI': 'TOTAL\nTRI',
        '% REAL X META': '% REAL X\nMETA'
})   


    # df_export['% REAL X\nMETA'] = df_export['% REAL X\nMETA'].apply(lambda x: round(x))


    display(df_export.head(1))

    if exportar:
        df_export.to_excel(fr'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\CAMPANHA - {nome_campanha}\consolidados gerados - para o envio\RESULTADO REPRESENTANTES\EXCEL NAO FORMATADO\{regiao}.xlsx', index=False)
        sleep(2)

,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,RODRIGO VIEIRA,18,ELIAS BATISTA,ITUIUTABA,7000,420.99,0.0,0.0,420.99,0.06


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,IRIS ROBERT,23,CARLOS ANTONIO DA SILVA,PATOS DE MINAS,2900,248.66,0.0,0.0,248.66,0.09


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,PAULO MIRANDA,9,GILMAR JACINTO,FRUTAL,9000,648.66,0.0,0.0,648.66,0.07


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,HELIO CAMPOS,377,EDUARDO BARBOSA,BARBACENA,3300,165.1,0.0,0.0,165.1,0.05


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,RONEY AGUIAR,257,CARLOS AFONSO DE ARAUJO,SÃO FRANCISCO,500,0.0,0.0,0.0,0.0,0.0


...

,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,MARCELO TOMAZELI,471,AILTON TRISTAO JUNIOR,BATATAIS,11000,1615.33,0.0,0.0,1615.33,0.15


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,CARLOS FERNANDES,574,JUCELINO FERREIRA,CAMPINAS,700,0.0,0.0,0.0,0.0,0.0


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,AMANCIO JUNIO,588,ALEX PEREIRA,CEILANDIA,800,19.67,0.0,0.0,19.67,0.02


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,LEANDRO JOSE,74,CAIRO VAZ,CALDAS NOVAS,5200,960.0,0.0,0.0,960.0,0.18


,SUPERVISOR,COD,NOME REPRESENTANTE,PRAÇA,META\nTRI,REAL\nOUTUBRO,REAL\nNOVEMBRO,REAL\nDEZEMBRO,TOTAL\nTRI,% REAL X\nMETA
0,LEANDRO QUEIROZ,432,GUSTAVO PITALUGA,GOIANIA,3500,208.0,0.0,0.0,208.0,0.06


### <span style="color: red;">**OPENPYXL**</span>

In [15]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl.worksheet.page import PageSetup
from openpyxl.worksheet.page import PageMargins

from openpyxl.styles import PatternFill, Border, Side, Alignment, Font, NamedStyle

#### **CONSOLIDADO POR REPRESENTANTE -> PARA OS SUPERVISORES**

In [16]:
caminho = rf'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\CAMPANHA - {nome_campanha}\consolidados gerados - para o envio\RESULTADO REPRESENTANTES\EXCEL NAO FORMATADO'
arquivos_excel = list(map(lambda x: ("\\").join([caminho, x]), os.listdir(caminho)))

# display(arquivos_excel)

In [17]:
cabecalho_planilha = f'CAMPANHA {nome_campanha} REPRESENTANTES - ARROZ - LINHA PREMIUM'

In [18]:
# MODELO
file = rf'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\CAMPANHA - {nome_campanha}\arq_excel\MODELO - RESULTADO REPRESENTANTES.xlsx'

wb_modelo = load_workbook(file)
ws_modelo = wb_modelo.active

In [19]:
thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

gray_fill = PatternFill(start_color="E7E6E6", end_color="E7E6E6", fill_type="solid")

yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

custom_number_format = NamedStyle(name="custom_number")
custom_number_format.number_format = '_-* #,##0_-;-* #,##0_-;_-* "-"??_-;_-@_-'

custom_percent_format = NamedStyle(name="custom_percent")
custom_percent_format.number_format = "0%"

##### **MAIN**

In [20]:
for arquivo in arquivos_excel:
    # CARREGANDO O ARQUIVO PRONTO
    wb_loaded = load_workbook(arquivo)
    ws_loaded = wb_loaded.active


    # CRIANDO UM NOVO ARQUIVO
    wb_new = Workbook()
    ws_new = wb_new.active


    # CABECALHO
    ws_new['A1'] = cabecalho_planilha

    # ADICIONANDO OS VALORES
    for row in ws_loaded.rows:

        for cell in row:
            numero_linha = cell.row
            letra_coluna = cell.column_letter
            valor = cell.value

            if numero_linha == 1:

                if letra_coluna in ['A', 'B', 'C', 'D']:
                    ws_new[f'{letra_coluna}4'] = valor

                elif letra_coluna in ['E', 'F', 'G', 'H']:
                    ws_new[f'{letra_coluna}3'] = valor
                    # ws_new[f'{letra_coluna}3'].alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')
                
                else:
                    colunas_restantes = {'I': 'J', 'J': 'K'}
                    ws_new[f'{colunas_restantes[letra_coluna]}3'] = valor
                    # ws_new[f'{colunas_restantes[letra_coluna]}3'].alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')
            
            else:

                if letra_coluna not in ['I', 'J']:
                    numero_linha += 3
                    ws_new[f'{letra_coluna}{numero_linha}'] = valor
                
                else:
                    colunas_restantes = {'I': 'J', 'J': 'K'}
                    numero_linha += 3
                    ws_new[f'{colunas_restantes[letra_coluna]}{numero_linha}'] = valor

    ultima_linha = ws_new.max_row
    penultima_linha = ultima_linha - 1

    column_letters = [get_column_letter(num_col) for num_col in range(1, ultima_linha + 1)]




    # ADICIONANDO FORMTAÇÃO
    ws_new.merge_cells('A1:K1')
    ws_new.merge_cells(f'A{penultima_linha}:D{penultima_linha}')
    ws_new.merge_cells(f'A{ultima_linha}:C{ultima_linha}')


    # FORMATAÇÃO DAS MERGEDCELLS
    # cabeçalho
    for cell in ws_new['A1:K1'][0]:
        cell.border = thin_border
        cell.font = Font(name = 'Courier New', bold = True, size = 22)


    for cell in ws_new[f'A{penultima_linha}:D{penultima_linha}'][0]:
        cell.border = thin_border
        cell.font = Font(name = 'Courier New', bold = True, size = 10)
        cell.alignment = Alignment(horizontal='center', vertical='center')


    for cell in ws_new[f'A{ultima_linha}:C{ultima_linha}'][0]:
        cell.border = thin_border
        cell.font = Font(name = 'Courier New', bold = True, size = 10)
        cell.alignment = Alignment(horizontal='center', vertical='center')



    for row in ws_new:
        for cell in row:
            numero_linha = cell.row
            # TRATANDO AS MERGEDCELLS
            try:
                letra_coluna = cell.column_letter
            except AttributeError:
                # cell.border = thin_border
                continue
            
            
            coluna_linha = f'{letra_coluna}{numero_linha}'
            valor = cell.value


            if type(valor) in (int, float):
                if letra_coluna != 'K':
                    cell.style = custom_number_format
                elif letra_coluna == 'K':
                    cell.style = custom_percent_format


            cell.fill = gray_fill
            cell.border = thin_border
            

            if numero_linha > 1:
                cell.font = Font(name = 'Courier New', bold = True, size = 10)
                
                
                if (valor == None):
                    cell.fill = PatternFill()
                    
                    if (numero_linha == 4) and (letra_coluna != 'I'):
                        cell.border = thin_border
                    else:  
                        cell.border = Border()
                        
                
                else:

                    if (4 < numero_linha < penultima_linha):
                        cell.fill = PatternFill()
                        cell.font = Font(name = 'Courier New', bold = False, size = 10)

                        if letra_coluna == 'E':
                            cell.fill = yellow_fill

                        elif letra_coluna == 'B':
                            cell.value = int(valor)
                            cell.alignment = Alignment(horizontal='center', vertical='center')


                    else:
                        cell.fill = gray_fill

                if numero_linha == 3:
                    cell.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')


    for letter in column_letters:
        ws_new.column_dimensions[letter].width = ws_modelo.column_dimensions[letter].width


    ws_new.row_dimensions[3].height = ws_modelo.row_dimensions[3].height

    ws_new.page_setup.orientation = 'landscape'
    ws_new.page_setup.paperSize = ws_new.PAPERSIZE_LETTER

    ws_new.page_margins = PageMargins(left=0.8, right=0.0, top=1.0, bottom=1.0, header=0.0, footer=0.0)

    ws_new.page_setup.scale = 74

    # ws_new.print_area = 'A1:D10'

    arquivo = arquivo.replace('EXCEL NAO FORMATADO', 'EXCEL FORMATADO')
    wb_new.save(arquivo)

#### **TESTE**

In [21]:
# MODELO

# file = r'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\TESTE\arq_excel\MODELO - RESULTADO REPRESENTANTES.xlsx'

# wb_modelo = load_workbook(file)
# ws_modelo = wb_modelo.active

In [22]:
# thin_border = Border(left=Side(style='thin'),
#                      right=Side(style='thin'),
#                      top=Side(style='thin'),
#                      bottom=Side(style='thin'))

# gray_fill = PatternFill(start_color="E7E6E6", end_color="E7E6E6", fill_type="solid")

# yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# custom_number_format = NamedStyle(name="custom_number")
# custom_number_format.number_format = '_-* #,##0_-;-* #,##0_-;_-* "-"??_-;_-@_-'

In [23]:
# ws_new.merge_cells('A1:K1')
# ws_new.merge_cells(f'A{penultima_linha}:D{penultima_linha}')
# ws_new.merge_cells(f'A{ultima_linha}:C{ultima_linha}')

# # FORMATAÇÃO DAS MERGEDCELLS
# # cabeçalho
# for cell in ws_new['A1:K1'][0]:
#     cell.border = thin_border
#     cell.font = Font(name = 'Courier New', bold = True, size = 22)


# for cell in ws_new[f'A{penultima_linha}:D{penultima_linha}'][0]:
#     cell.border = thin_border
#     cell.font = Font(name = 'Courier New', bold = True, size = 10)
#     cell.alignment = Alignment(horizontal='center', vertical='center')

# for cell in ws_new[f'A{ultima_linha}:C{ultima_linha}'][0]:
#     cell.border = thin_border
#     cell.font = Font(name = 'Courier New', bold = True, size = 10)
#     cell.alignment = Alignment(horizontal='center', vertical='center')



# for row in ws_new:
#     for cell in row:
#         numero_linha = cell.row
#         # TRATANDO AS MERGEDCELLS
#         try:
#             letra_coluna = cell.column_letter
#         except AttributeError:
#             # cell.border = thin_border
#             continue
        
        
#         coluna_linha = f'{letra_coluna}{numero_linha}'
#         valor = cell.value


#         if type(valor) == int:
#             cell.style = custom_number_format
#             # print(valor)


#         cell.fill = gray_fill
#         cell.border = thin_border
        

#         if numero_linha > 1:
#             cell.font = Font(name = 'Courier New', bold = True, size = 10)
            
            
#             if (valor == None):
#                 cell.fill = PatternFill()
                
#                 if (numero_linha == 4) and (letra_coluna != 'I'):
#                     cell.border = thin_border
#                 else:  
#                     cell.border = Border()
                    
            
#             else:

#                 if (4 < numero_linha < penultima_linha):
#                     cell.fill = PatternFill()
#                     cell.font = Font(name = 'Courier New', bold = False, size = 10)

#                     if letra_coluna == 'E':
#                         cell.fill = yellow_fill


#                 else:
#                     cell.fill = gray_fill

#             if numero_linha == 3:
#                 cell.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')

In [24]:
# for letter in column_letters:
#     ws_new.column_dimensions[letter].width = ws_modelo.column_dimensions[letter].width


# ws_new.row_dimensions[3].height = ws_modelo.row_dimensions[3].height

In [25]:
# ws_new.merge_cells(start_row=1, start_column=1, end_row=1, end_column=11)
# ws_new.merge_cells(f'A{penultima_linha}:D{penultima_linha}')
# ws_new.merge_cells(f'A{ultima_linha}:C{ultima_linha}')

In [26]:
# wb_new.save('TESTE.xlsx')

# os.startfile('TESTE.xlsx')

### <span style="color: red;">**TRANSFORMANDO OS ARQUIVOS EM PDF**</span>

##### **SALVANDO AS ATUALIZAÇÕES DO CONSOLIDADO DE GERENTES E DIRETORES ANTES DE TRASNFORMAR EM PDF**

In [27]:
file_path = os.getcwd() + r'\consolidados gerados - para o envio\RESULTADO PARCIAL - AREA - REGIAO.xlsx'

# Abre o Excel invisivelmente e atualiza os links
with xw.App(visible=True) as app:
    # Abre a pasta de trabalho
    wb = app.books.open(file_path)

    # Atualiza todos os links
    wb.api.UpdateLink(Name=wb.fullname, Type=1)  # Type=1 é para links de arquivos externos

    # Salva as alterações
    wb.save()

    # Fecha a pasta de trabalho
    wb.close()

##### **PARCIAL PARA OS SUPERVISORES**

In [28]:
arquivos_excel = os.listdir(rF'D:\Perfil\Desktop\PROJETOS EMPRESA\NEI\CAMPANHA\CAMPANHA - {nome_campanha}\consolidados gerados - para o envio\RESULTADO REPRESENTANTES\EXCEL FORMATADO')

# print(arquivos_excel)

for nome_arquivo_xlsx in arquivos_excel:
    nome_arquivo_pdf = nome_arquivo_xlsx.removesuffix('.xlsx')


    # Defina o caminho do arquivo XLSX recebido e o caminho de saída para o PDF
    caminho_xlsx = os.getcwd() + '\consolidados gerados - para o envio\RESULTADO REPRESENTANTES\EXCEL FORMATADO' + fr'\{nome_arquivo_xlsx}'
    caminho_pdf = os.getcwd() + '\consolidados gerados - para o envio\PDFS PRONTOS\SEPARADOS' + fr'\{nome_arquivo_pdf} PARCIAL.pdf'


    # Abre o Excel e o arquivo
    app = xw.App(visible=False)  # Mantém o Excel oculto durante o processo
    wb = app.books.open(caminho_xlsx)

    wb.to_pdf(caminho_pdf)

    wb.close()
    app.quit()

##### **PARCIAL PARA OS GERENTES**

In [29]:
caminho_xlsx = os.getcwd() + r'\consolidados gerados - para o envio\RESULTADO PARCIAL - AREA - REGIAO.xlsx'
caminho_pdf = os.getcwd() + r'\consolidados gerados - para o envio\PDFS PRONTOS\JUNTOS\CONSOLIDADO - GERENTE.pdf'


app = xw.App(visible=False)
wb = app.books.open(caminho_xlsx)

ws = wb.sheets['REGIOES - GERENTES']

ws.to_pdf(caminho_pdf)

wb.close()
app.quit()

if abrir_pdfs:
    os.startfile(caminho_pdf)

##### **PARCIAL PARA DIRETORIA**

In [30]:
caminho_xlsx = os.getcwd() + r'\consolidados gerados - para o envio\RESULTADO PARCIAL - AREA - REGIAO.xlsx'
caminho_pdf = os.getcwd() + r'\consolidados gerados - para o envio\PDFS PRONTOS\SEPARADOS\CONSOLIDADO - DIRETORIA.pdf'


app = xw.App(visible=False)
wb = app.books.open(caminho_xlsx)

ws = wb.sheets['AREAS - DIRETORIA']

ws.to_pdf(caminho_pdf)

wb.close()
app.quit()

if abrir_pdfs:
    os.startfile(caminho_pdf)